# II. 신경망 만들기
***

In [1]:
pwd

'/home/myhwang/deep-learning-from-scratch-3'

In [3]:
from dezero import *
import contextlib
import numpy as np
import graphviz
import dezero.functions as F

## 1. 기존 패키지가 텐서자료에도 적용 되는가?

스칼라

In [ ]:
x=Variable(np.array(1.))
y=F.sin(x)
print(y)

variable(0.8414709848078965)


행렬

In [ ]:
x=Variable(np.array([[1,2,3],[4,5,6]]))
y=F.sin(x)
print(y)

variable([[ 0.84147098  0.90929743  0.14112001]
          [-0.7568025  -0.95892427 -0.2794155 ]])


행렬 + 스칼라

In [ ]:
x=Variable(np.array([[1,2,3],[4,5,6]]))
c=Variable(np.array([[10,20,30],[40,50,60]]))
y=x+c
print(y)

variable([[11 22 33]
          [44 55 66]])


## 2. 형상변환 함수

reshape 함수 구현

In [4]:
x=np.array([[1,2,3],[4,5,6]])
y=np.reshape(x,(6,))

print(y)

[1 2 3 4 5 6]


In [4]:
# class Reshape(Function):
#     def __init__(self, shape):
#         self.shape = shape

#     def forward(self, x):
#         self.x_shape = x.shape
#         y = x.reshape(self.shape)
#         return y

#     def backward(self, gy):
#         return reshape(gy, self.x_shape)


# def reshape(x, shape):
#     if x.shape == shape:
#         return as_variable(x)
#     # f=Reshape(shape)
#     # return f(x)
#     return Reshape(shape)(x)

x=Variable(np.array([[1,2,3],[4,5,6]]))
y=F.reshape(x,(6,))
y.backward(retain_grad=True)

print(y)
print(x.grad)

variable([1 2 3 4 5 6])
variable([[1 1 1]
          [1 1 1]])


편의성 패치

In [8]:
#numpy의 reshape
x=np.random.rand(1,2,3)

y=x.reshape((2,3)) #튜플로 받기
y=x.reshape([2,3]) #리스트로 받기
y=x.reshape(2,3) #인수를 풀어서 받기

In [4]:
# # reshape 입력 편의성을 위한 코드 core.py의 class Variable에 추가
# def reshape(self, *shape):
#     if len(shape) == 1 and isinstance(shape[0], (tuple, list)):
#         shape = shape[0]
#     return dezero.funtions.reshape(self, shape)

x=Variable(np.random.rand(1,2,3))

y=x.reshape((2,3)) #튜플로 받기
y=x.reshape([2,3]) #리스트로 받기
y=x.reshape(2,3) #인수를 풀어서 받기

transpose 함수 구현

In [6]:
x=np.array([[1,2,3],[4,5,6]])
y=np.transpose(x)

print(y)

[[1 4]
 [2 5]
 [3 6]]


In [ ]:
# class Transpose(Function):
#     def forward(self, x):
#         y = np.transpose(x)
#         return y

#     def backward(self, gy):
#         gx = transpose(gy)
#         return gx


# def transpose(x):
#     f = Transpose()
#     return f(x)

x=Variable(np.array([[1,2,3],[4,5,6]]))
y=F.transpose(x)
y.backward(retain_grad=True)

print(y)
print(x.grad)

variable([1 2 3 4 5 6])
variable([[1 1 1]
          [1 1 1]])


편의성 패치

In [21]:
# # transpose 입력 편의성을 위한 코드 core.py의 class Variable에 추가
# def transpose(self):
#     return dezero.functions.transpose(self)
#
# @property
# def T(self):
#     return dezero.functions.transpose(self)

x=Variable(np.random.rand(2,3))

y=x.transpose() 
y=x.T()
y=x.T # @property

y=F.transpose(x)
y.data
print(x.data.shape, y.shape)


(2, 3) (3, 2)


데이터 구조 순서 변경

numpy

In [6]:
A, B, C, D =1,2,3,4
x=np.random.rand(A,B,C,D)
y=x.transpose(1,0,3,2)

print(x)
print(x.shape)
print('\n')
print(y)
print(y.shape)

[[[[0.62492053 0.34837898 0.08979581 0.16201332]
   [0.66160309 0.93933077 0.68230613 0.92872977]
   [0.07879896 0.04240806 0.52253779 0.81951213]]

  [[0.28829795 0.59486033 0.75372237 0.45591727]
   [0.51402042 0.75263629 0.16519023 0.15963545]
   [0.23740602 0.18873385 0.7978057  0.82055843]]]]
(1, 2, 3, 4)


[[[[0.62492053 0.66160309 0.07879896]
   [0.34837898 0.93933077 0.04240806]
   [0.08979581 0.68230613 0.52253779]
   [0.16201332 0.92872977 0.81951213]]]


 [[[0.28829795 0.51402042 0.23740602]
   [0.59486033 0.75263629 0.18873385]
   [0.75372237 0.16519023 0.7978057 ]
   [0.45591727 0.15963545 0.82055843]]]]
(2, 1, 4, 3)


In [ ]:
# # transpose 구조 순서 변경을 위한 코드 functions.py의 class 수정
# class Transpose(Function):
#     def __init__(self, axes=None):
#         self.axes = axes

#     def forward(self, x):
#         y = x.transpose(self.axes)
#         return y

#     def backward(self, gy):
#         if self.axes is None:
#             return transpose(gy)

#         axes_len = len(self.axes)
#         inv_axes = tuple(np.argsort([ax % axes_len for ax in self.axes]))
#         return transpose(gy, inv_axes)
#
# def transpose(x, axes=None):
#     return Transpose(axes)(x)



# # transpose 구조 순서 변경을 위한 코드 core.py의 class Variable에 수정
#
# def transpose(self, *axes):
#     if len(axes) == 0:
#         axes = None
#     elif len(axes) == 1:
#         if isinstance(axes[0], (tuple, list)) or axes[0] is None:
#             axes = axes[0]
#     return dezero.functions.transpose(self, axes)

In [6]:
A, B, C, D =1,2,3,4
x=Variable(np.random.rand(A,B,C,D))
y=x.transpose(1,0,3,2)

print(x.data)
print(x.shape)
print('\n')
print(y.data)
print(y.shape)


[[[[0.33495746 0.99285864 0.64429329 0.73883322]
   [0.91049988 0.29999847 0.75331492 0.16324055]
   [0.79819818 0.08766936 0.32783892 0.26212729]]

  [[0.94827607 0.51457012 0.84246428 0.71420237]
   [0.41365796 0.46076555 0.48962624 0.43056031]
   [0.96835464 0.60271868 0.71956376 0.70118926]]]]
(1, 2, 3, 4)


[[[[0.33495746 0.91049988 0.79819818]
   [0.99285864 0.29999847 0.08766936]
   [0.64429329 0.75331492 0.32783892]
   [0.73883322 0.16324055 0.26212729]]]


 [[[0.94827607 0.41365796 0.96835464]
   [0.51457012 0.46076555 0.60271868]
   [0.84246428 0.48962624 0.71956376]
   [0.71420237 0.43056031 0.70118926]]]]
(2, 1, 4, 3)


In [1]:
from dezero import *
import contextlib
import numpy as np
import graphviz
import dezero.functions as F